In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, LSTM, concatenate, Dropout
from tensorflow.keras.utils import plot_model
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import RandomizedSearchCV
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint

# seed
import os
seed = 123
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

In [3]:
image_generator = ImageDataGenerator(width_shift_range=0.1,
                                     height_shift_range=0.1, 
                                     zoom_range=[0.8,1.2],
                                     #brightness_range=[0.75,1.25], 
                                     shear_range=10)

In [4]:
x1 = train.drop(['id', 'digit', 'letter'], axis=1).values
x1 = x1.reshape(-1, 28, 28, 1)
x1 = x1/255
x1_remake = []
for i in range(x1.shape[0]):
    num_aug = 0
    tmp = x1[i]
    tmp = tmp.reshape((1,) + tmp.shape)
    for x_aug in image_generator.flow(tmp, batch_size = 1) :
        if num_aug >= 1:
            break
        x1_remake.append(x_aug[0])
        num_aug += 1
x1_remake = np.array(x1_remake)

x1_total = np.concatenate((x1, x1_remake), axis=0)
print(x1_total.shape)

(4096, 28, 28, 1)


In [5]:
y1_data = train['digit']
y1 = np.zeros((len(y1_data), len(y1_data.unique())))
for i, digit in enumerate(y1_data):
    y1[i, digit] = 1

y1_remake = y1.copy()
y1_total = np.concatenate((y1, y1_remake), axis=0)
print(y1_total.shape)

(4096, 10)


In [6]:
x1_let = train['letter'].values
x1_let = x1_let[:, np.newaxis]
en = OneHotEncoder()
x1_let = en.fit_transform(x1_let).toarray()

x1_remake_let = x1_let.copy()

x1_letter_total = np.concatenate((x1_let, x1_remake_let), axis=0)
x1_letter_total.shape

(4096, 26)

In [7]:
x1_train, x1_val, y1_train, y1_val = train_test_split(x1_total, y1_total, test_size=0.2, shuffle=True, stratify=y1_total)

print(x1_train.shape)
print(x1_val.shape)
print(y1_train.shape)
print(y1_val.shape)

x1_letter_train = x1_letter_total[:x1_train.shape[0],:]
x1_letter_val = x1_letter_total[x1_train.shape[0]:,:]
print(x1_letter_train.shape)
print(x1_letter_val.shape)

(3276, 28, 28, 1)
(820, 28, 28, 1)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [8]:
x2_train = np.reshape(x1_train, (x1_train.shape[0], x1_train.shape[1], x1_train.shape[2]))
x2_val = np.reshape(x1_val, (x1_val.shape[0], x1_val.shape[1], x1_val.shape[2]))
y2_train = y1_train.copy()
y2_val = y1_val.copy()
x2_letter_train = x1_letter_train.copy()
x2_letter_val = x1_letter_val.copy()

print(x2_train.shape)
print(x2_val.shape)
print(y2_train.shape)
print(y2_val.shape)
print(x2_letter_train.shape)
print(x2_letter_val.shape)

(3276, 28, 28)
(820, 28, 28)
(3276, 10)
(820, 10)
(3276, 26)
(820, 26)


In [9]:
input1 = Input(shape=(28,28,1))
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(input1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(64, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Conv2D(128, (2,2), activation='relu', padding='same')(x1)
x1 = Dropout(0.2)(x1)
x1 = MaxPooling2D((2,2))(x1)
x1 = Flatten()(x1)

input2 = Input(shape=(26,))
merge1 = concatenate([x1, input2])

x2 = Dense(500, activation='relu')(merge1)
x2 = Dropout(0.2)(x2)
x2 = Dense(100, activation='relu')(x2)

input3 = Input(shape=(28,28))
x3 = LSTM(512, activation='relu', return_sequences=True)(input3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(256, activation='relu', return_sequences=True)(x3)
x3 = Dropout(0.2)(x3)
x3 = LSTM(128, activation='relu')(x3)
x3 = Dropout(0.2)(x3)
x3 = Dense(100, activation='relu')(x3)

input4 = Input(shape=(26,))
merge2 = concatenate([x3, input4])

x4 = Dense(100, activation='relu')(merge2)

merge = concatenate([x2, x4])

x5 = Dense(100, activation='relu')(merge)
x5 = Dropout(0.2)(x5)
x5 = Dense(50, activation='relu')(x5)

x5 = Dropout(0.2)(x5)
output1 = Dense(10, activation='softmax')(x5)

x6 = Dense(50, activation='relu')(x5)
output2 = Dense(10, activation='softmax')(x6)

model = Model(inputs = [input1, input2, input3, input4], outputs = [output1, output2])
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 28, 28, 1)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 28, 28, 64)   320         input_1[0][0]                    
__________________________________________________________________________________________________
dropout (Dropout)               (None, 28, 28, 64)   0           conv2d[0][0]                     
__________________________________________________________________________________________________
max_pooling2d (MaxPooling2D)    (None, 14, 14, 64)   0           dropout[0][0]                    
______________________________________________________________________________________________

In [10]:
model.compile(optimizer = 'adam', metrics = ['accuracy'], loss = 'categorical_crossentropy', 
              loss_weights = [0.9, 0.1])
history = model.fit([x1_train, x1_letter_train, x2_train, x2_letter_train], [y1_train, y2_train],
                    validation_data=([x1_val, x1_letter_val, x2_val, x2_letter_val], [y1_val, y2_val]), 
                    batch_size=64, epochs=50, verbose=1, 
                    callbacks = [ModelCheckpoint('./models/{epoch:02d}-{val_dense_6_accuracy:.4f}.h5',
                    monitor='val_dense_6_accuracy', verbose=1, save_best_only=True, mode='auto')])

Train on 3276 samples, validate on 820 samples
Epoch 1/50
3264/3276 [============================>.] - ETA: 0s - loss: 2.3042 - dense_6_loss: 2.3041 - dense_8_loss: 2.3051 - dense_6_accuracy: 0.1112 - dense_8_accuracy: 0.1029
Epoch 00001: val_dense_6_accuracy improved from -inf to 0.11220, saving model to ./models/01-0.1122.h5
3276/3276 [==============================] - 35s 11ms/sample - loss: 2.3041 - dense_6_loss: 2.3037 - dense_8_loss: 2.3045 - dense_6_accuracy: 0.1117 - dense_8_accuracy: 0.1035 - val_loss: 2.2987 - val_dense_6_loss: 2.2985 - val_dense_8_loss: 2.3002 - val_dense_6_accuracy: 0.1122 - val_dense_8_accuracy: 0.1171
Epoch 2/50
3264/3276 [============================>.] - ETA: 0s - loss: 2.2382 - dense_6_loss: 2.2340 - dense_8_loss: 2.2759 - dense_6_accuracy: 0.1562 - dense_8_accuracy: 0.1290
Epoch 00002: val_dense_6_accuracy improved from 0.11220 to 0.21463, saving model to ./models/02-0.2146.h5
3276/3276 [==============================] - 30s 9ms/sample - loss: 2.2374 

Epoch 15/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.5982 - dense_6_loss: 0.5963 - dense_8_loss: 0.6156 - dense_6_accuracy: 0.8088 - dense_8_accuracy: 0.8033
Epoch 00015: val_dense_6_accuracy did not improve from 0.71585
3276/3276 [==============================] - 31s 9ms/sample - loss: 0.5980 - dense_6_loss: 0.5952 - dense_8_loss: 0.6151 - dense_6_accuracy: 0.8089 - dense_8_accuracy: 0.8034 - val_loss: 0.9124 - val_dense_6_loss: 0.9114 - val_dense_8_loss: 0.9316 - val_dense_6_accuracy: 0.7000 - val_dense_8_accuracy: 0.6939
Epoch 16/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.5347 - dense_6_loss: 0.5333 - dense_8_loss: 0.5465 - dense_6_accuracy: 0.8229 - dense_8_accuracy: 0.8183
Epoch 00016: val_dense_6_accuracy improved from 0.71585 to 0.71829, saving model to ./models/16-0.7183.h5
3276/3276 [==============================] - 31s 9ms/sample - loss: 0.5339 - dense_6_loss: 0.5297 - dense_8_loss: 0.5418 - dense_6_accuracy: 0.8230 - dense_8_acc

Epoch 30/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.1839 - dense_6_loss: 0.1834 - dense_8_loss: 0.1891 - dense_6_accuracy: 0.9436 - dense_8_accuracy: 0.9409
Epoch 00030: val_dense_6_accuracy did not improve from 0.76341
3276/3276 [==============================] - 32s 10ms/sample - loss: 0.1845 - dense_6_loss: 0.1862 - dense_8_loss: 0.1925 - dense_6_accuracy: 0.9435 - dense_8_accuracy: 0.9408 - val_loss: 0.9282 - val_dense_6_loss: 0.9236 - val_dense_8_loss: 0.9765 - val_dense_6_accuracy: 0.7244 - val_dense_8_accuracy: 0.7171
Epoch 31/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.1746 - dense_6_loss: 0.1738 - dense_8_loss: 0.1819 - dense_6_accuracy: 0.9442 - dense_8_accuracy: 0.9412
Epoch 00031: val_dense_6_accuracy did not improve from 0.76341
3276/3276 [==============================] - 34s 10ms/sample - loss: 0.1749 - dense_6_loss: 0.1751 - dense_8_loss: 0.1850 - dense_6_accuracy: 0.9438 - dense_8_accuracy: 0.9408 - val_loss: 0.8787 - val_de

Epoch 45/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.0946 - dense_6_loss: 0.0934 - dense_8_loss: 0.1048 - dense_6_accuracy: 0.9688 - dense_8_accuracy: 0.9657
Epoch 00045: val_dense_6_accuracy did not improve from 0.76341
3276/3276 [==============================] - 34s 10ms/sample - loss: 0.0951 - dense_6_loss: 0.0962 - dense_8_loss: 0.1073 - dense_6_accuracy: 0.9686 - dense_8_accuracy: 0.9655 - val_loss: 0.9765 - val_dense_6_loss: 0.9622 - val_dense_8_loss: 1.0542 - val_dense_6_accuracy: 0.7512 - val_dense_8_accuracy: 0.7500
Epoch 46/50
3264/3276 [============================>.] - ETA: 0s - loss: 0.0989 - dense_6_loss: 0.0980 - dense_8_loss: 0.1066 - dense_6_accuracy: 0.9709 - dense_8_accuracy: 0.9669
Epoch 00046: val_dense_6_accuracy did not improve from 0.76341
3276/3276 [==============================] - 34s 10ms/sample - loss: 0.0991 - dense_6_loss: 0.0990 - dense_8_loss: 0.1081 - dense_6_accuracy: 0.9707 - dense_8_accuracy: 0.9667 - val_loss: 0.9580 - val_de

In [14]:
evaluate = model.evaluate([x1_val, x1_letter_val, x2_val, x2_letter_val], [y1_val, y2_val], 
                          batch_size=64, verbose=2)
evaluate

820/1 - 2s - loss: 0.8184 - dense_6_loss: 0.9595 - dense_8_loss: 1.0310 - dense_6_accuracy: 0.7646 - dense_8_accuracy: 0.7659


[0.9710885992864283, 0.95954156, 1.0309701, 0.76463413, 0.76585364]

In [11]:
#from tensorflow.keras.models import load_model
#best_model = load_model('./models/35-0.6695.h5')

In [12]:
x1_test = test.drop(['id', 'letter'], axis=1).values
x1_test = x1_test.reshape(-1, 28, 28, 1)
x1_test = x1_test/255

x2_test = test.drop(['id', 'letter'], axis=1).values
x2_test = x2_test.reshape(-1, 28, 28)
x2_test = x2_test/255

x1_letter_test = test['letter'].values
x1_letter_test = x1_letter_test[:, np.newaxis]
en = OneHotEncoder()
x1_letter_test = en.fit_transform(x1_letter_test).toarray()

x2_letter_test = x1_letter_test.copy()

y1_test, y2_test = model.predict([x1_test, x1_letter_test, x2_test, x2_letter_test])
y_1 = np.argmax(y1_test, axis=1)
y_2 = np.argmax(y2_test, axis=1)
print(y_1)
print(y_2)

[6 9 2 ... 6 1 0]
[6 9 2 ... 6 1 0]


In [15]:
submission = pd.read_csv('data/submission.csv')
submission['digit'] = y_1
submission.to_csv('submission(val7646).csv', index=False)